In [1]:
import numpy as np
from matplotlib import pyplot as plt

from qepy.calculator import QEpyCalculator 
from qepy.io import QEInput
from qepy.driver import Driver

from ase.build import bulk
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal
from ase import Atoms
import ase.io

try:
    from mpi4py import MPI
    comm=MPI.COMM_WORLD
except:
    comm=None

qe_options = {
        '&control': {
            'calculation': "'scf'",
            'prefix': "'si'",
            'pseudo_dir': "'/home/valeria/Desktop/programs/dftpy/examples/ofpp/EAC/upf/blps/'",
            'restart_mode': "'from_scratch'"},
        '&system': {
            'ibrav' : 1,
            'nat': 2,
            'degauss': 0.005,
            'ecutwfc': 60,
            'occupations': "'smearing'"},
        'atomic_species': ['Si  28.08 si.lda.upf'],
        'cell_parameters angstrom': ['5.43    0.0000000000000000    0.0000000000000003',
                                     '0.0000000000000009    5.43    0.0000000000000003',
                                     '0.0000000000000000    0.0000000000000000    5.43'],
         'k_points automatic': ['20 20 20 1 1 1']}

# KS DFT
l = np.linspace(0.8, 1.4, 30)
# ks_ke = []
for i in np.arange(0, len(l), 1):
    n = int(i)
    X = l[i]
    inputfile = 'bct5.vasp'
    si = ase.io.read(inputfile, format='vasp')

    cell = si.get_cell()
    si.set_cell(cell * X, scale_atoms=True)
    qe_options = QEInput.update_atoms(si, qe_options = qe_options,  extrapolation=False)
    QEInput().write_qe_input("/home/valeria/Documents/DFTPY/cWT-KEDF/Phases/Model_ML_function/Ks_calculations/DATA_bct5/Si_bct5"+str(n)+".in", qe_options=qe_options)
    driver = Driver('/home/valeria/Documents/DFTPY/cWT-KEDF/Phases/Model_ML_function/Ks_calculations/DATA_bct5/Si_bct5'+str(n)+'.in', comm=comm, logfile='/home/valeria/Documents/DFTPY/cWT-KEDF/Phases/Model_ML_function/Ks_calculations/DATA_bct5/Si_bct5'+str(n)+'.out')
    driver.scf()
    driver.calc_energy()
    D = driver.get_output()
    rho = driver.data2field(driver.get_density())
    ions = driver.get_dftpy_ions()
    rho.write('/home/valeria/Documents/DFTPY/cWT-KEDF/Phases/Model_ML_function/Ks_calculations/DATA_bct5/rho0_bct5_'+str(n)+'.xsf', ions=ions)
    driver.stop()

/home/valeria/anaconda3/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')
